# Data Analysis
## Tweet data

In [62]:
import sqlite3
import pandas as pd
from afinn import Afinn
import json

# Fetch data from db
db = sqlite3.connect('../tweets.db')
c = db.cursor()

In [63]:
c.execute("SELECT * FROM tweets")

rows = c.fetchall()

tweets = pd.DataFrame(columns=['id', 'date', 'json', 'filter'], data=rows)
tweets['date'] = pd.to_datetime(tweets['date'],format='%Y-%m-%d %H:%M:%S')

In [64]:
afinn = Afinn()

def tweet_text(data):
    info = json.loads(data)
    if 'extended_tweet' in info:
        text = info['extended_tweet']['full_text']
    else:
        text = info['text']
    return text

def sentiment_score(data):
    return afinn.score(tweet_text(data))

def is_retweet(data):
    info = json.loads(data)
    return 'retweeted_status' in info

def tweet_symbols(data):
    info = json.loads(data)
    return info['entities']['symbols']

def is_multiple_cashtag(tweet):
    info = json.loads(tweet)
    return len(info['entities']['symbols'])>1

In [66]:
tweets['sentiment_score'] = tweets['json'].apply(lambda x: sentiment_score(x))
tweets['text'] = tweets['json'].apply(lambda x: tweet_text(x))
tweets['retweet'] = tweets['json'].apply(lambda x: is_retweet(x))
tweets['symbols'] = tweets['json'].apply(lambda x: tweet_symbols(x))

In [74]:
pd.options.display.max_colwidth = 3000
tweets[['sentiment_score','text','retweet']][tweets['sentiment_score']>0].sample(5)


,sentiment_score,text,retweet
86,2.0,"this channel called $DGB before it made 6x. they only share high reward, low risk signals. join:\n\nhttps://t.co/w1TEIU3QMf\n\n$BTC $ETH $ETC $BCH $LTC $XRP $DASH $BTG $XLM $XMR $ZEC $ADA $SNT $NEO $NXT $OMG $POWR $VTC $LUX $XEM $LSK $DGB $DOGE $XVG $GRS $AMP $ZCL $DRGN $DCR 18030",False
44,8.0,"RT @cryptopedians: today is the deadline of 35 $ltc telegram giveaway!!!! \n\nwinner will also be announced soon, stay tuned if you wanna win…",True
472,3.0,"RT @CryptoPunish: #Giveaway of the week! .2 BTC (Currently worth $2,080.71 USD) Please make sure you follow me &amp; RT this to enter. I’ll be…",True
137,3.0,RT @TraderCobb: $BTC ### I AM VERY EXCITED TO ANNOUNCE ###....... The https://t.co/9LT8ldTQU0 website is ready for launch. All courses and…,True
57,8.0,"RT @cryptopedians: We passed 4250 mark with 2 more days to go , wow!!! \n\nGood news is that You can still join the 35 $ltc telegram giveaway…",True


In [75]:
tweets[['retweet','text','symbols']].sample(5)

,retweet,text,symbols
53,True,"RT @emilybary: “Bitcoin, for us, is not stopping at buying and selling,” Square CEO Jack Dorsey said. $BTC $SQ https://t.co/FhLKfKOz9C","[{'text': 'BTC', 'indices': [102, 106]}, {'text': 'SQ', 'indices': [107, 110]}]"
331,False,#BTC price decreased! Buying @ $10512.37 &amp; Selling @ $10512.37 $btc $eth $etc $neo $dgb $dash $strat $eos $omg $pay $xrp $xmr $gas 😝,"[{'text': 'btc', 'indices': [68, 72]}, {'text': 'eth', 'indices': [73, 77]}, {'text': 'etc', 'indices': [78, 82]}, {'text': 'neo', 'indices': [83, 87]}, {'text': 'dgb', 'indices': [88, 92]}, {'text': 'dash', 'indices': [93, 98]}, {'text': 'strat', 'indices': [99, 105]}, {'text': 'eos', 'indices': [106, 110]}, {'text': 'omg', 'indices': [111, 115]}, {'text': 'pay', 'indices': [116, 120]}, {'text': 'xrp', 'indices': [121, 125]}, {'text': 'xmr', 'indices': [126, 130]}, {'text': 'gas', 'indices': [131, 135]}]"
524,False,"this channel called $DGB before it made 6x. they only share high reward, low risk signals. join:\n\nhttps://t.co/gdeJWJuRPa\n\n$BTC $ETH $ETC $BCH $LTC $XRP $DASH $BTG $XLM $XMR $ZEC $ADA $SNT $NEO $NXT $OMG $POWR $VTC $LUX $XEM $LSK $DGB $DOGE $XVG $GRS $AMP $ZCL $DRGN $DCR 28518","[{'text': 'DGB', 'indices': [20, 24]}]"
47,False,Price action for $BTC on Bitfinex \n\nIn the last 5 minutes:\nBUY: $10621.00 SELL: $10600.00 \nBulls bought 120.683346 BTC \nBears sold -552.677327 BTC,"[{'text': 'BTC', 'indices': [17, 21]}]"
248,True,"RT @PeterLBrandt: Many historic bull markets have been comprised by parabolic surges followed by deep prolonged corrections, then the next…",[]


## Processed Data with different time resolution

In [9]:
c.execute("SELECT * FROM vart_10min")

rows = c.fetchall()
varT= pd.DataFrame(columns=['date', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13'], data=rows)

In [10]:
varT.tail()

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
84,2018-02-28 08:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
85,2018-02-28 08:40:00,108,15,5,58,49,12,2,45,26,11,71,15,43
86,2018-02-28 08:50:00,167,18,18,67,55,17,4,82,45,16,106,26,51
87,2018-02-28 09:00:00,197,31,16,65,48,4,1,112,56,28,113,30,48
88,2018-02-28 09:10:00,111,17,16,53,35,18,8,57,44,10,57,26,27
